<a href="https://colab.research.google.com/github/2303A53017/AIML-Lab/blob/main/lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
class Game:
    def actions(self, state):
        """Return a collection of the allowable moves from this state."""
        raise NotImplementedError

    def result(self, state, move):
        """Return the state that results from making a move from a state."""
        raise NotImplementedError

    def is_terminal(self, state):
        """Return True if this is a final state for the game."""
        return not self.actions(state)

    def utility(self, state, player):
        """Return the value of this final state to player."""
        raise NotImplementedError

In [6]:
def play_game(game, strategies: dict, verbose=False):
    """Play a turn-taking game.

    'strategies' is a {player name: function} dict,
    where each function(state, game) is used to get the player’s move.
    """
    state = game.initial
    while not game.is_terminal(state):
        player = state.to_move
        move = strategies[player](game, state)
        state = game.result(state, move)
        if verbose:
            print('Player', player, 'move:', move)
            print(state)
    return state

In [7]:
def play_game(game, strategies: dict, verbose=False):
    """Play a turn-taking game.

    'strategies' is a {player name: function} dict,
    where each function(state, game) is used to get the player’s move.
    """
    state = game.initial
    while not game.is_terminal(state):
        player = state.to_move
        move = strategies[player](game, state)
        state = game.result(state, move)
        if verbose:
            print('Player', player, 'move:', move)
            print(state)
    return state

In [8]:
import math

def alphabeta_search(game, state):
    """Search game to determine best action using alpha-beta pruning.
    Search all the way to the leaves.
    """
    player = state.to_move

    def max_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = -math.inf, None
        for a in game.actions(state):
            v2, _ = min_value(game.result(state, a), alpha, beta)
            if v2 > v:
                v, move = v2, a
            alpha = max(alpha, v)
            if v >= beta:
                return v, move  # Beta cutoff
        return v, move

    def min_value(state, alpha, beta):
        if game.is_terminal(state):
            return game.utility(state, player), None
        v, move = math.inf, None
        for a in game.actions(state):
            v2, _ = max_value(game.result(state, a), alpha, beta)
            if v2 < v:
                v, move = v2, a
            beta = min(beta, v)
            if v <= alpha:
                return v, move  # Alpha cutoff
        return v, move

    return max_value(state, -math.inf, math.inf) if player == 'max' else min_value(state, -math.inf, math.inf)

In [9]:
class TicTacToe(Game):
    """Play TicTacToe on a 'height' by 'width' board, needing 'k' in a row to win.
    'X' plays first against 'O'."""

    def __init__(self, height=3, width=3, k=3):
        self.k = k  # k in a row to win
        self.squares = {(x, y) for x in range(width) for y in range(height)}  # All possible squares
        self.initial = Board(height=height, width=width, to_move='X', utility=0)

    def actions(self, board):
        """Legal moves are any square not yet taken."""
        return self.squares - set(board)  # Returns available squares not already taken

    def result(self, board, square):
        """Place a marker for the current player on square."""
        player = board.to_move
        # Create a new board with the square occupied by the current player
        board = board.new({square: player}, to_move=('O' if player == 'X' else 'X'))
        # Check if the move results in a win
        win = self.k_in_row(board, player, square, self.k)
        # Update utility: +1 for X win, -1 for O win, 0 otherwise
        board.utility = 0 if not win else (+1 if player == 'X' else -1)
        return board

    def utility(self, board, player):
        """Return the value to player; 1 for win, -1 for loss, 0 otherwise."""
        return board.utility if player == 'X' else -board.utility

    def is_terminal(self, board):
        """A board is a terminal state if it is won or there are no empty squares."""
        return board.utility != 0 or len(self.squares) == len(board)

    def display(self, board):
        """Display the current board."""
        print(board)

    def k_in_row(self, board, player, square, k):
        """Return True if player has k pieces in a line through square."""
        x, y = square

        def in_row(x, y, dx, dy):
            """Count consecutive pieces of player starting from (x, y) in direction (dx, dy)."""
            if (x, y) not in board or board[x, y] != player:
                return 0
            return 1 + in_row(x + dx, y + dy, dx, dy)

        # Check all 4 directions (vertical, horizontal, two diagonals)
        return any(
            in_row(x, y, dx, dy) + in_row(x - dx, y - dy, -dx, -dy) - 1 >= k
            for (dx, dy) in [(0, 1), (1, 0), (1, 1), (1, -1)]
        )

In [10]:
from collections import defaultdict

class Board(defaultdict):
    """A board has the player to move, a cached utility value,
    and a dict of {(x, y): player} entries, where player is 'X' or 'O'."""

    empty = '.'
    off = '#'

    def __init__(self, width=8, height=8, to_move=None, **kwds):
        """Initialize the board with a given width, height, and the player to move."""
        super().__init__(lambda: self.empty)  # Default value for empty squares
        self.width = width
        self.height = height
        self.to_move = to_move
        self.update(kwds)  # Update the board with any additional keyword arguments

    def new(self, changes: dict, **kwds) -> 'Board':
        """Given a dict of {(x, y): contents} changes, return a new Board with the changes."""
        board = Board(width=self.width, height=self.height, to_move=self.to_move, **kwds)
        board.update(self)  # Copy the current board's state
        board.update(changes)  # Apply changes to the new board
        return board

    def missing(self, loc):
        """Return the default value for a location that's off the board."""
        x, y = loc
        if 0 <= x < self.width and 0 <= y < self.height:
            return self.empty
        else:
            return self.off

    def __hash__(self):
        """Generate a hash value for the board, based on its contents and the current player."""
        return hash(tuple(sorted(self.items()))) + hash(self.to_move)

    def __repr__(self):
        """Return a string representation of the board."""
        def row(y):
            return ' '.join(self[x, y] for x in range(self.width))
        return '\n'.join(map(row, range(self.height))) + '\n'

In [12]:
import random

def random_player(game, state):
    """A game player who chooses a random legal move."""
    return random.choice(list(game.actions(state)))

def player(search_algorithm):
    """A game player who uses the specified search algorithm.

    The search algorithm should return a (value, move) pair, and this player will select the move.
    """
    return lambda game, state: search_algorithm(game, state)[1]